In [120]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
from proj1_helpers import *
from helpers import *
from ml_functions import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [2]:
DATA_TRAIN_PATH = '../Data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Change -999s to median of column (feature) and standardize

In [3]:
tX.shape

(250000, 30)

In [ ]:
for col in tX.T:
    med = np.median(col[col != -999])
    col[col == -999] = med

## Remove column 22 for use a model index for multiple model case

In [ ]:
#keep model index column seperate
model_index = tX[:, 22]

#remove model index column from tx
tX2 = np.delete(tX, 22, 1)
    
print(tX2.shape)

## Build Polyonmial Matrix for 4 model

In [8]:
degree = 2
#polyX = np.full((tX.shape[0], tX.shape[1] * (degree+1)), 0.0)
polyX = list()
for row in tX2:
    polyX.append(build_poly2(row, degree))

PolyXNP = np.array(polyX)
PolyXNP2 = PolyXNP.reshape((PolyXNP.shape[0], PolyXNP.shape[2]))

#Standardize
PolyXNP2 = standardize(PolyXNP2)
PolyXNP2 = np.hstack((np.ones((PolyXNP2.shape[0],1)), PolyXNP2))

In [ ]:
PolyXNP2

## Training Single model

In [61]:
tX4 = standard2(tX)

tXSingleTestOnly = np.hstack((np.ones((tX.shape[0],1)), tX4))
tXSingleTestOnly.shape

(250000, 31)

In [64]:
y[y == -1] = 0

In [107]:
y[y == 1].shape

(85667,)

In [128]:
gamma = 0.0000002
lambda_ = 0.01
max_iters = 10000

initial_w = np.zeros((tXSingleTestOnly.shape[1], 1))

losses, w = reg_logistic_regression_test(y, tXSingleTestOnly, lambda_, initial_w, max_iters, gamma)

In [143]:
w.shape

(31, 1)

In [130]:
y_pred = sigmoid(np.dot(tXSingleTestOnly, w))
y_pred[y_pred > 0.5].shape

(52825,)

In [91]:
y_pred

array([[ 0.22315644],
       [ 0.29686668],
       [-0.11854201],
       ..., 
       [ 0.28013609],
       [ 0.38984445],
       [ 0.23547771]])

## Training with 4 models:

In [205]:
Fused = np.c_[y, PolyXNP2]

In [165]:
Fused[:,5]

array([ 0.00464148,  0.00520637,  0.0079336 , ...,  0.00267873,
        0.0021645 ,  0.00230664])

In [446]:
Fused_0 = Fused[model_index == 0.0]
Fused_1 = Fused[model_index == 1.0]
Fused_2 = Fused[model_index == 2.0]
Fused_3 = Fused[model_index == 3.0]

gamma = 0.000000005
lambda_ = 0.01
max_iters = 300

loss1, w1 = pen_logistic_regression_test(Fused_0[:,0], Fused_0[:,1:Fused_0.shape[1]], lambda_, gamma, max_iters)
loss2, w2 = pen_logistic_regression_test(Fused_1[:,0], Fused_1[:,1:Fused_1.shape[1]], lambda_, gamma, max_iters)
loss3, w3 = pen_logistic_regression_test(Fused_2[:,0], Fused_2[:,1:Fused_2.shape[1]], lambda_, gamma, max_iters)
loss4, w4 = pen_logistic_regression_test(Fused_3[:,0], Fused_3[:,1:Fused_3.shape[1]], lambda_, gamma, max_iters)

#weights = ridge_regression(y, tX, 0.01)

weights = (w1, w2, w3, w4)

#w = logistic_regression(y,tX,gamma,max_iters)

(99913, 59)
(99913,)
(59, 1)
[-45062.96678364]
(77544, 59)
(77544,)
(59, 1)
[-46246.10469503]
(50379, 59)
(50379,)
(59, 1)
[-35208.7823107]
(22164, 59)
(22164,)
(59, 1)
[-14088.95896855]


In [450]:
w3

array([[  6.81000640e-04],
       [  1.75434470e-04],
       [ -1.00380667e-04],
       [ -1.03889103e-03],
       [ -1.83943166e-04],
       [ -5.46980441e-05],
       [ -5.84469312e-05],
       [  3.44737414e-04],
       [  3.16619405e-05],
       [  5.97276403e-03],
       [  4.46729048e-03],
       [  2.38242374e-03],
       [  6.88586941e-04],
       [ -2.37516441e-03],
       [ -2.22599176e-03],
       [ -2.33347898e-04],
       [ -5.98702456e-04],
       [ -5.63192275e-05],
       [ -4.01072574e-07],
       [  5.97863361e-04],
       [  1.38698720e-04],
       [ -5.56188745e-04],
       [ -1.13249818e-04],
       [  7.55604335e-03],
       [  8.33594150e-03],
       [  1.06574608e-02],
       [  1.04020830e-02],
       [  5.30304787e-04],
       [  5.10825168e-05],
       [  1.45945856e-04],
       [ -6.07518500e-04],
       [  2.29077758e-04],
       [  8.87424315e-05],
       [ -1.02566795e-04],
       [ -3.06482316e-05],
       [  1.63544730e-04],
       [ -6.91904229e-04],
 

In [451]:
weights2 = (w1.reshape((w1.shape[0],)), w2.reshape((w2.shape[0],)),w3.reshape((w3.shape[0],)), w4.reshape((w4.shape[0],)))


## Generate predictions and save ouput in csv format for submission:

In [133]:
DATA_TEST_PATH = '../Data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

### Prepare Test Data Multiple

In [132]:
for col in tX_test.T:
    med = np.median(col[col != -999])
    col[col == -999] = med

#keep model index column seperate
model_index2 = tX_test[:, 22]

#remove model index column from tx
tX_test = np.delete(tX_test, 22, 1)

#Use list to calculate polynomial faster
polyXTest = list()
for row in tX_test:
    polyXTest.append(build_poly2(row, degree))

PolyXNPTest = np.array(polyXTest)
PolyXNPTest2 = PolyXNPTest.reshape((PolyXNPTest.shape[0], PolyXNPTest.shape[2]))

#Standardize
#tX_test, mean_x, std_x = standardize(tX_test)
PolyXNPTest2 = standard2(PolyXNPTest2)
PolyXNPTest2 = np.hstack((np.ones((PolyXNPTest2.shape[0],1)), PolyXNPTest2))


PolyXNPTest2.shape

NameError: name 'tX_test' is not defined

In [452]:
OUTPUT_PATH = '../Data/resultLogisticPenalized6.csv' 
# TODO: fill in desired name of output file for submission
y_pred = predict_labels3(weights2, PolyXNPTest2, degree, model_index2)
print(y_pred.shape)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

[ 0.55204022  0.52927842  0.55873225 ...,  0.55711393  0.52899992
  0.56392765]
(568238,)


In [145]:
y_pred

array([[ 1.],
       [ 1.],
       [ 1.],
       ..., 
       [ 1.],
       [ 1.],
       [ 1.]])

## Prepare test data Single

In [146]:
tX_test_single = tX_test
for col in tX_test_single.T:
    med = np.median(col[col != -999])
    col[col == -999] = med
    
tX_test_single = standard2(tX_test_single)

tX_test_single = np.hstack((np.ones((tX_test_single.shape[0],1)), tX_test_single))

print(tX_test_single.shape)


(568238, 31)


In [148]:
OUTPUT_PATH = '../Data/resultLogisticPenalizedSingleModel2.csv' 
# TODO: fill in desired name of output file for submission
y_pred = predict_labels_logistic_single(w, tX_test_single)
print(y_pred.shape)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

[[ 0.18008755]
 [ 0.21627501]
 [ 0.32692703]
 ..., 
 [ 0.50223199]
 [ 0.44951093]
 [ 0.1534008 ]]
(568238, 1)


In [150]:
y_pred[y_pred<0.5]

array([-1., -1., -1., ..., -1., -1., -1.])